In [1]:
data_var = '2024-01-28'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13296,2024-01-28,Espanha Acb,08:30,Basquet Girona,Granada,1.60,2.36,166.5,1.86,1.94,-4.5,2.02,1.77,CS1iXEd6,0.625000,0.423729,0.537634,0.515464,0.048729,0.8,0.2,NaN,NaN,206.916,58.725506,0.283813,258.194,141.797006,0.549188,164.008,404.564,118.44,327.24,0.0,0.0,0.0,0.0,0.271415,0.029773,0.093286,-1.95,-0.390,-1.538462,0.000000,0.0,0.000000,2.16,0.432,3.148148,0.000000,0.0,0.000000
13297,2024-01-28,Espanha Acb,09:00,Gran Canaria,Real Madrid,2.81,1.44,165.5,1.86,1.94,4.5,1.99,1.81,QV5eWYBC,0.355872,0.694444,0.537634,0.515464,0.050316,0.6,0.6,NaN,NaN,174.478,71.600806,0.410372,117.902,39.378824,0.333996,112.944,120.514,121.03,81.74,0.0,0.0,0.0,0.0,0.455876,0.029773,0.066989,-0.16,-0.032,-56.562500,0.000000,0.0,0.000000,0.55,0.110,4.000000,0.000000,0.0,0.000000
13298,2024-01-28,Espanha Acb,13:00,Barcelona,Palencia,1.04,11.50,164.5,1.82,1.96,-18.5,2.08,1.68,xxC37KJP,0.961538,0.086957,0.549451,0.510204,0.048495,0.4,1.0,NaN,NaN,117.250,13.082140,0.111575,230.304,86.128131,0.373976,101.848,419.898,99.36,252.84,0.0,0.0,0.0,0.0,1.179639,0.052378,0.150448,-0.44,-0.088,-0.454545,0.000000,0.0,0.000000,-2.06,-0.412,-25.485437,0.000000,0.0,0.000000
13299,2024-01-28,Espanha Acb,13:00,Baskonia,Obradoiro CAB,1.21,4.53,169.5,1.87,1.92,-11.5,2.09,1.69,rJlW2vSt,0.826446,0.220751,0.534759,0.520833,0.047197,1.0,0.6,NaN,NaN,159.234,32.783143,0.205880,295.742,242.693322,0.820625,133.088,347.960,137.16,172.42,0.0,0.0,0.0,0.0,0.817977,0.018657,0.149652,-0.06,-0.012,-17.500000,0.000000,0.0,0.000000,1.82,0.364,9.697802,0.000000,0.0,0.000000
13300,2024-01-28,Espanha Acb,14:30,Valencia,Unicaja,1.74,2.11,160.5,1.85,1.93,-3.5,2.08,1.71,M9mz2bsm,0.574713,0.473934,0.540541,0.518135,0.048646,0.8,0.4,NaN,NaN,175.464,137.041068,0.781021,137.490,39.201065,0.285119,101.730,120.848,104.72,124.74,0.0,0.0,0.0,0.0,0.135911,0.029930,0.138063,-2.61,-0.522,-1.417625,0.000000,0.0,0.000000,0.83,0.166,6.686747,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13482,2024-01-28,Eua Nba,16:00,Detroit Pistons,Oklahoma City Thunder,6.79,1.13,238.5,1.88,2.04,11.5,2.10,1.83,0C6QxZjh,0.147275,0.884956,0.531915,0.490196,0.032231,0.4,0.6,NaN,NaN,582.154,281.257079,0.483132,195.374,42.434287,0.217195,587.664,240.464,197.75,214.00,0.0,0.0,0.0,0.0,1.010663,0.057723,0.097160,-3.25,-0.650,-8.907692,0.319942,0.2,-0.119942,2.24,0.448,0.290179,0.530039,0.6,0.069961
13483,2024-01-28,Mundo Liga Dos Campeões Das Américas,21:10,Nacional,São Paulo,3.12,1.33,162.5,1.83,1.83,2.5,2.01,1.68,b7v7EL9m,0.320513,0.751880,0.546448,0.546448,0.072393,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,281.58,274.30,0.0,0.0,0.0,0.0,0.568863,0.000000,0.126474,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
13484,2024-01-28,Chile Lnb,19:00,Puerto Montt,Deportes Castro,2.01,1.73,149.5,1.83,1.87,-1.5,1.86,1.84,dUnn6qkO,0.497512,0.578035,0.546448,0.534759,0.075547,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,494.73,0.00,1.0,0.0,0.0,0.0,0.105877,0.015289,0.007644,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
13485,2024-01-28,Chile Lnb,20:00,CD Puerto Varas,Ancud,3.25,1.31,160.5,1.85,1.85,6.5,1.95,1.72,WzseQnzt,0.307692,0.763359,0.540541,0.540541,0.071051,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.00

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:00,Eua Nba,Orlando Magic,Phoenix Suns,228.5,1.86,0.9983,Over
1,01:59,Eua Ncaa,Hawaii,Cal Poly,126.5,1.95,0.7542,Over
2,16:00,Eua Nba,Detroit Pistons,Oklahoma City Thunder,238.5,1.88,1.0000,Over
